In [16]:
import sys, os
sys.path.append("../..")
sys.path.append("..")
sys.path.append(os.getcwd())

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import copy

import random
from numpy.linalg import svd, matrix_rank, norm
from sklearn import linear_model
import pickle

In [135]:

def mse(y, y_pred):
    # y, y_pred are df
    return ((y - y_pred) ** 2).mean(axis=1)

def rmse(y, y_pred):
    # y, y_pred are df
    return np.sqrt(((y - y_pred) ** 2).mean(axis=1))

def mape_vector(y, y_pred):
    # y, y_pred are vector
    mask = (y != 0)
    return np.mean(np.abs((y - y_pred)[mask] / y[mask]))

def mape(y, y_pred):
    # y, y_pred are df
    n = y.shape[0]
    keys = y.index
    df_return = pd.DataFrame(index = keys, columns =["MAPE"])
    for i in range(n):
        y1 = y.iloc[i,:]
        y_pred1 = y_pred.iloc[i,:]
        df_return.loc[keys[i],] = mape_vector(y1, y_pred1)
    return df_return


In [153]:
def mergeDFlist(list1, list2):
    merged_list = []
    n = len(list1)
    for i in range(n):
        merged = pd.concat([list1[i],list2[i]], axis=0)
        merged_list.append(merged)
    return merged_list
        
class analysis:
    def __init__(self, df_true, df_pred_list, T_0s):
        self.true = df_true
        self.pred = df_pred_list
        self.T_0s = T_0s
        self.mse_list = []
        self.rmse_list = []
        self.mape_list = []
    
    def run(self):
        self.mse_list = []
        self.rmse_list = []
        self.mape_list = []
        for i in range(len(T_0s)):
            T_0 = T_0s[i]
            y = self.true.iloc[:,T_0:]
            y_pred = self.pred[i].iloc[:,T_0:T]

            df_mse = mse(y,y_pred)
            df_rmse = rmse(y,y_pred)
            df_mape = mape(y,y_pred)
            self.mse_list.append(df_mse)
            self.rmse_list.append(df_rmse)
            self.mape_list.append(df_mape)    

            print("*** Training Percentage", train_pcts[i], "***")
            print("MSE : ", df_mse.mean())
            print("RMSE: ", df_rmse.mean())    
            print("MAPE: ",df_mape.mean().values[0])
            print()
        

# No HSVT; saeyoung code

In [8]:
# experiment prarams
train_pcts = [0.1, 0.25, 0.5, 0.75, 0.9]
freq = 15
T = int(12*60*4/freq + 1)
T_0s = [int(np.ceil(train_pct * T)) for train_pct in train_pcts]
singvals= 0
donor_ids = np.array(pd.read_pickle('../data/nba-hosoi/donor_ids.pkl'))
target_ids = np.array(pd.read_pickle('../data/nba-hosoi/target_ids.pkl'))
metrics = ['points','assists', 'rebounds', 'bs', 'fouls']

# true
home_true = pd.read_pickle("../data/nba-hosoi/target_home_points.pkl")
away_true = pd.read_pickle("../data/nba-hosoi/target_away_points.pkl")

# pred
filename = "result_team/saeyoung_no_hsvt_home.pkl"
with open(filename, 'rb') as f:
    home_pred = pickle.load(f)

filename = "result_team/saeyoung_no_hsvt_away.pkl"
with open(filename, 'rb') as f:
    away_pred = pickle.load(f)

In [154]:
all_true = pd.concat([home_true, away_true], axis=0)
all_pred = mergeDFlist(home_pred, away_pred)

separateMetricsNohsvt = analysis(all_true, all_pred, T_0s)
separateMetricsNohsvt.run()

*** Training Percentage 0.1 ***
MSE :  176.38353359882913
RMSE:  11.18965799905146
MAPE:  0.168116352563

*** Training Percentage 0.25 ***
MSE :  96.1340477988557
RMSE:  8.393512293339903
MAPE:  0.106177314813

*** Training Percentage 0.5 ***
MSE :  48.57258800022953
RMSE:  6.060265983840488
MAPE:  0.0636765918395

*** Training Percentage 0.75 ***
MSE :  21.10106327347633
RMSE:  4.045669730598443
MAPE:  0.0375101177523

*** Training Percentage 0.9 ***
MSE :  8.175506299348267
RMSE:  2.535312519562961
MAPE:  0.0219144096288



In [155]:
print("Home")
print()
separateMetricsNohsvt_home = analysis(home_true, home_pred, T_0s)
separateMetricsNohsvt_home.run()

Home

*** Training Percentage 0.1 ***
MSE :  172.8030911140695
RMSE:  11.011161987675438
MAPE:  0.160459733214

*** Training Percentage 0.25 ***
MSE :  94.75524330483589
RMSE:  8.357583858509754
MAPE:  0.103639649543

*** Training Percentage 0.5 ***
MSE :  47.76372961102372
RMSE:  6.027934116701378
MAPE:  0.062069777998

*** Training Percentage 0.75 ***
MSE :  20.598504047495954
RMSE:  3.9907256077795354
MAPE:  0.0357845167923

*** Training Percentage 0.9 ***
MSE :  8.112435907711168
RMSE:  2.5232694782365286
MAPE:  0.021384878722



In [156]:
print("Away")
print()
separateMetricsNohsvt_away = analysis(away_true, away_pred, T_0s)
separateMetricsNohsvt_away.run()

Away

*** Training Percentage 0.1 ***
MSE :  179.96397608358896
RMSE:  11.368154010427443
MAPE:  0.168116352563

*** Training Percentage 0.25 ***
MSE :  97.51285229287477
RMSE:  8.429440728170055
MAPE:  0.106177314813

*** Training Percentage 0.5 ***
MSE :  49.381446389435055
RMSE:  6.092597850979625
MAPE:  0.0636765918395

*** Training Percentage 0.75 ***
MSE :  21.603622499456737
RMSE:  4.1006138534173395
MAPE:  0.0375101177523

*** Training Percentage 0.9 ***
MSE :  8.23857669098537
RMSE:  2.547355560889395
MAPE:  0.0219144096288

